In [ ]:
import evaluate
import pandas as pd
import glob
import ast
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load generated answers

def process_files_to_aggregated_dfs(directory_path):
    # initialize dicts to store QAs
    data_dicts = {
        "qa_eval_set": {},
        "chat_gpt_questions": {},
        "unrelated_topic_questions": {}
    }

    file_pattern = f"{directory_path}/query_response_pairs_hybrid_search*.txt"

    for filename in glob.glob(file_pattern):
        with open(filename, 'r') as file:
            contents = file.read()
            data_list = ast.literal_eval(contents)
            
            # extract unique id from filename
            file_id = os.path.basename(filename).split('.')[0].split('_')[5:]

            if len(data_list) >= 17:
                # Process for qa_eval_set
                for pair in data_list[:10]:  # First 10 for qa_eval_set
                    query, answer = pair
                    if query not in data_dicts["qa_eval_set"]:
                        data_dicts["qa_eval_set"][query] = {"query": query}
                    data_dicts["qa_eval_set"][query][f"answer with hybrid search weights: {file_id[0]}, {file_id[1]}"] = answer
                
                # Process for chat_gpt_questions
                for pair in data_list[10:15]:  # Next 5 for chat_gpt_questions
                    query, answer = pair
                    if query not in data_dicts["chat_gpt_questions"]:
                        data_dicts["chat_gpt_questions"][query] = {"query": query}
                    data_dicts["chat_gpt_questions"][query][f"answer with hybrid search weights: {file_id[0]}, {file_id[1]}"] = answer
                
                # Process for unrelated_topic_questions
                for pair in data_list[-2:]:  # Last 2 for unrelated_topic_questions
                    query, answer = pair
                    if query not in data_dicts["unrelated_topic_questions"]:
                        data_dicts["unrelated_topic_questions"][query] = {"query": query}
                    data_dicts["unrelated_topic_questions"][query][f"answer with hybrid search weights: {file_id[0]}, {file_id[1]}"] = answer

    # Convert dictionaries to DataFrames
    for category, data_dict in data_dicts.items():
        data_dicts[category] = pd.DataFrame.from_dict(data_dict, orient='index').fillna('').reset_index(drop=True)

    return data_dicts["qa_eval_set"], data_dicts["chat_gpt_questions"], data_dicts["unrelated_topic_questions"]

directory_path = './results/hybrid_search'
qa_eval_set_df, chat_gpt_qa_df, unrelated_topic_qa_df = process_files_to_aggregated_dfs(directory_path)


In [ ]:
print("QA Eval Set:")
#pd.set_option('display.max_colwidth', None)
print(len(qa_eval_set_df))
qa_eval_set_df.head(1)

In [ ]:
print("\nChatGPT QA:")
print(len(chat_gpt_qa_df))
#chat_gpt_qa_df.head(1)

In [ ]:
print("\nUnrelated Topic QA:")
print(len(unrelated_topic_qa_df))
#unrelated_topic_qa_df.head(1)

In [ ]:
# Extract queries
qa_eval_queries = qa_eval_set_df["query"].to_list()
# chat_gpt_queries = chat_gpt_qa_df["query"].to_list()
# unrelated_queries = unrelated_topic_qa_df["query"].to_list()


In [ ]:
# Load gold truth answers
qa_set = pd.read_csv("../../data/questions_answers/questions_answers.csv")
gold_truth_qa = qa_set[["QUESTION", "LONG_ANSWER"]][qa_set['QUESTION'].isin(qa_eval_queries)]

gold_answers = gold_truth_qa["LONG_ANSWER"].to_list()

gold_truth_qa

**ℹ️ One of the tested questions is unfortunately not available in the QA evaluation set, thus only 9 of our 10 asked question can be evaluated using ground truth results.**

➡️ "What are the effects of α1-antitrypsin (AAT) treatment on chronic fatigue syndrome (CFS) based on a case study involving a 49-year-old woman?" will be deleted.

In [ ]:
generated_answers_list = [
    qa_eval_set_df["answer with hybrid search weights: 0, 1"].to_list(),
    qa_eval_set_df["answer with hybrid search weights: 01, 09"].to_list(),
    qa_eval_set_df["answer with hybrid search weights: 02, 08"].to_list(),
    qa_eval_set_df["answer with hybrid search weights: 03, 07"].to_list(),
    qa_eval_set_df["answer with hybrid search weights: 04, 06"].to_list(),
    qa_eval_set_df["answer with hybrid search weights: 05, 05"].to_list(),
    qa_eval_set_df["answer with hybrid search weights: 06, 04"].to_list(),
    qa_eval_set_df["answer with hybrid search weights: 07, 03"].to_list(),
    qa_eval_set_df["answer with hybrid search weights: 08, 02"].to_list(),
    qa_eval_set_df["answer with hybrid search weights: 09, 01"].to_list(),
    qa_eval_set_df["answer with hybrid search weights: 1, 0"].to_list()
]

In [ ]:
# Load metrics
bleu = evaluate.load('bleu')
rouge = evaluate.load('rouge')
bertscore = evaluate.load("bertscore")

list_names = ["0.0, 1.0", "0.1, 0.9", "0.2, 0.8", "0.3, 0.7", "0.4, 0.6", "0.5, 0.5", "0.6, 0.4", "0.7, 0.3", "0.8, 0.2", "0.9, 0.1", "1.0, 0.0" ]

def compute_scores(gen_answers_list, gold_truths, names):
    scores = []
    
    for name, gen_answers in zip(names, gen_answers_list):
        gen_answers_copy = gen_answers[:]
        del gen_answers_copy[8]

        bleu_score = bleu.compute(predictions=gen_answers_copy, references=gold_truths)
        rouge_score = rouge.compute(predictions=gen_answers_copy, references=gold_truths)
        bert = bertscore.compute(predictions=gen_answers_copy, references=gold_truths, lang="en")
        bertscore_averaged = {}

        for key in bert.keys():
            if key!='hashcode':
                bertscore_averaged[key]=np.mean(bert[key])

        # save results to file
        with open("results/hybrid_search/bleu_rouge_bert_scores.txt", "a") as file:
            file.write(f"Results for hybrid search weights: {name}\n")
            file.write(f"BLEU Score: {bleu_score}\n")
            file.write(f"ROUGE Score: {rouge_score}\n")
            file.write(f"BERTScore Averaged: {bertscore_averaged}\n\n")

        # return scores for plotting
        scores.append({
            'name': name,
            'bleu': bleu_score,
            'rouge': rouge_score,
            'bertscore': bertscore_averaged
        })

    return scores

scores = compute_scores(generated_answers_list, gold_answers, list_names)

In [ ]:
# Plot results
weights = [score['name'] for score in scores]
bleu_scores = [score['bleu']['bleu'] for score in scores]
rouge1_scores = [score['rouge']['rouge1'] for score in scores]
bertscore_f1_scores = [score['bertscore']['f1'] for score in scores]

# BLEU scores
plt.figure(figsize=(14, 7))
plt.subplot(3, 1, 1)  # 3 rows, 1 column, 1st subplot
plt.plot(weights, bleu_scores, marker='o', linestyle='-', color='blue', label='BLEU Score')
plt.title('Evaluation Scores Across Different Hybrid Search Weights')
plt.xlabel('Hybrid Search Weights')
plt.ylabel('BLEU Score')
plt.xticks(rotation=45)
plt.grid(True)

# ROUGE-1 scores
plt.subplot(3, 1, 2)  # 3 rows, 1 column, 2nd subplot
plt.plot(weights, rouge1_scores, marker='s', linestyle='--', color='red', label='ROUGE-1 Score')
plt.xlabel('Hybrid Search Weights')
plt.ylabel('ROUGE-1 Score')
plt.xticks(rotation=45)
plt.grid(True)

# BERTScore F1 scores
plt.subplot(3, 1, 3)  # 3 rows, 1 column, 3rd subplot
plt.plot(weights, bertscore_f1_scores, marker='^', linestyle='-.', color='green', label='BERTScore F1')
plt.xlabel('Hybrid Search Weights')
plt.ylabel('BERTScore F1')
plt.xticks(rotation=45)
plt.grid(True)

plt.tight_layout()

# Save plot
plt.savefig("../../docs/images/bleu_rouge_bert_hybrid_search.png", dpi=150, bbox_inches='tight')
plt.show()